# Looking at the data

In [6]:
import pandas as pd
submissions = pd.read_csv("sel_hn_stories1.csv")
print(submissions.shape)
print(submissions.columns)
print(submissions.head(5))
submissions = submissions.dropna()

(3107, 6)
Index([u'Unnamed: 0', u'Unnamed: 0.1', u'submission_time', u'upvotes', u'url',
       u'headline'],
      dtype='object')
   Unnamed: 0  Unnamed: 0.1       submission_time  upvotes  \
0      249371        249371  2014-04-16T05:26:14Z        1   
1     1457172       1457172  2009-02-19T01:13:54Z        7   
2      338241        338241  2013-12-15T00:57:32Z        9   
3     1238163       1238163  2010-12-30T07:00:09Z       14   
4      708856        708856  2012-09-20T13:55:31Z        1   

                             url  \
0  signup.containerizetheweb.com   
1                 techcrunch.com   
2                            NaN   
3              computerworld.com   
4        keepitmovingfitness.com   

                                            headline  
0  Show HN: Checkout 'Containerize the Web' Site ...  
1     Coghead Grinds To A Halt Heads To The Deadpool  
2           Ask HN: Toxic companies like Booking.com  
3  Today's $60 1TB drive would have cost $1 trill...  
4  

# Tokenization

In [7]:
tokenized_headlines = []
for item in submissions["headline"]:
    tokenized_headlines.append(item.split(" "))
print(tokenized_headlines)

[['Show', 'HN:', 'Checkout', "'Containerize", 'the', "Web'", 'Site', '(Docker', 'Resource)'], ['Coghead', 'Grinds', 'To', 'A', 'Halt', 'Heads', 'To', 'The', 'Deadpool'], ["Today's", '$60', '1TB', 'drive', 'would', 'have', 'cost', '$1', 'trillion', 'in', 'the', "'50s"], ['How', 'I', 'Became', 'Selfish', 'and', 'Changed', 'My', 'LIFE'], ['I', "Can't", 'Wait', 'for', 'NoSQL', 'to', 'Die'], ['Amazon', 'Web', 'Services', 'Is', 'Going', 'On', 'A', 'Sales', 'Hiring', 'Binge'], ['After', '2', 'Years', 'of', 'Testing', 'Venmo', 'Opens', 'Payment', 'Service', 'to', 'Public'], ['IDV', 'User', 'Experience:', 'CraigsZips'], ['Steve', 'Jobs', 'Preparing', 'for', 'First', 'TV', 'Appearance', '(1978)', '(97', 'Seconds)'], ['Search', 'Engine', 'That', 'Finds', 'Internet-Connected', 'Cameras', 'Traffic', 'Lights'], ['The', 'rise', 'of', 'the', 'mobile', 'app', 'user', 'over', 'the', 'website', 'user.'], ['Pep', 'talk', 'for', 'new', 'Ph.D.', 'students'], ['Consejos', 'para', 'elegir', 'las', 'cortinas',

# Preprocessing

In [8]:
punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
clean_tokenized = []
for item in tokenized_headlines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, "")
        tokens.append(token)
    clean_tokenized.append(tokens)

# Assembling a matrix

In [9]:
import numpy as np
unique_tokens = []
single_tokens = []
for tokens in clean_tokenized:
    for token in tokens:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token in single_tokens and token not in unique_tokens:
            unique_tokens.append(token)

counts = pd.DataFrame(0, index=np.arange(len(clean_tokenized)), columns=unique_tokens)

# Counting tokens

In [10]:

for i, item in enumerate(clean_tokenized):
    for token in item:
        if token in unique_tokens:
            counts.iloc[i][token] += 1

# Removing extraneous columns 




In [11]:
word_counts = counts.sum(axis=0)

counts = counts.loc[:,(word_counts >= 5) & (word_counts <= 100)]

# Splitting the data



In [12]:


from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, submissions["upvotes"], test_size=0.2, random_state=1)


# Making predictions

In [13]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)


# Calculating error 

In [14]:
mse = sum((y_test - predictions) ** 2) / len(predictions)

In [16]:
print(mse)

4520.41213511
